In [1]:
import pandas as pd
from tsfresh import  select_features
from datetime import date
import numpy as np
from tsfresh.utilities.dataframe_functions import impute
from datetime import datetime,timedelta
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from tqdm import tqdm

from utils.constants import Constants
from utils.tracker import init_logging
from classifier.phcp import phcp
from sklearn import datasets
import time

d:\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
d:\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
d:\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
d:\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with

Create label and combine the separate data

In [2]:

def get_days(year, month, day):
    get_day = date(year, month, day)
    return int(get_day.strftime('%j'))  # %j represents the jth day of the year 
def get_label_shift(year,days,gap_hour,size_hour,shift_hour):
    if(year%4==0):
        day=366
    else:
        day=365

    shift_windows=int(shift_hour/gap_hour)
    windows=int((day*24-size_hour)/gap_hour)
    windows=windows+shift_windows

    df=pd.read_csv("new_data.csv",index_col =0)
    df['A'] = pd.to_datetime(df['A']) # date转为时间格式
    df_data=df.values
    data_year=[x for x in df_data if x[0].year==year or x[0].year==year+1]
    label=np.zeros(windows).astype(np.int32)

    begin_time=str(year)+"-01-01-00"
    label_t=[]
    begin_time=datetime.strptime(begin_time, '%Y-%m-%d-%H')
    label_t.append(begin_time+timedelta(hours=shift_hour))
    #label_t.append(begin_time)
    for i in range(0,windows-1):
        label_t.append(label_t[-1]+timedelta(hours=gap_hour))
    label_t=np.array(label_t)


    label_len=len(label)
    for i in data_year:
        dd = i[0]
        if(dd.year==year):
            day=get_days(dd.year, dd.month, dd.day)
            offset=(day-1)*12+int(dd.hour/gap_hour)+1-1-shift_windows #1st begin with 0, int(dd.hour/gap_hour)+1-1 is the hour offset, +1:the remaining time, -1:begin with 0
        elif(dd.year==year+1):
            offset=-1
            if(day==1 and dd.hour<shift_hour):
                offset=(day-1)*12+int(dd.hour/gap_hour)+1-1
                offset=label_len-shift_windows+offset
        scale=i[1]
        if offset<label_len and offset>=0:
            if(scale=="S" and label[offset]<1):
                
                label[offset]=1
            elif(scale=="M" ):
                label[offset]=2
            elif(scale=="L"):
                label[offset]=3

    #relabel
    size_windows=int(size_hour/gap_hour)
    for i in range(0,len(label)-size_windows):
        label[i]=max(label[i:i+size_windows])




    temp=np.empty(shape=(0,0))
    temp_t=np.empty(shape=(0,0))
    
    for day in days:
        begin=int((day[0]-1)*(24/gap_hour))
        length=int(((day[1]-day[0]+1)*24-size_hour)/gap_hour)
        temp=np.append(temp,label[begin:begin+length])
        temp_t=np.append(temp_t,label_t[begin:begin+length])
    total = np.array([temp,temp_t]).T
        
    # result=pd.DataFrame(temp,columns=["label"]) 
    result=pd.DataFrame(total,columns=["label","time"]) 

    return result

In [29]:
if __name__ == '__main__':
    gap=4
    size=12
    sampling_rate=0.01
    prefix="./hf_result_save/"+str(gap)+"_"+str(size)+"_"+str(sampling_rate)+"_"

    shift=6
    X=pd.DataFrame()
    y=pd.DataFrame() 
    suffix="_WEST.csv"
    days_years=[[2013,[[141,365]]],[2014,[[1,365]]],[2015,[[1,171],[182,283]]],[2016,[[19,366]]],[2017,[[1,15],[41,151]]]]
    for i in days_years:
        filename=prefix+str(i[0])+suffix 
        x_temp=pd.read_csv(filename,index_col=0)
        X=pd.concat([X,x_temp]) 
        y_temp=get_label_shift(i[0],i[1],gap,size,shift)
        y=pd.concat([y,y_temp]) 

    prefix="./hf_data_save/"+str(gap)+"_"+str(size)+"_"+str(sampling_rate)+"_"
    y.reset_index(drop=True, inplace=True)
    
    impute(X)
    X.reset_index(drop=True, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)
    X_train = select_features(X_train, y_train['label'], ml_task='classification', n_jobs=4)  # or boruta
    X = X[X_train.columns]
    X.to_csv(prefix+"X.csv",index=True)
    
    y.to_csv(prefix+"y.csv",index=True)

d:\anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['HHZ.D__query_similarity_count__query_None__threshold_0.0'
 'HHN.D__query_similarity_count__query_None__threshold_0.0'
 'HHE.D__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


Additional Attribute

In [5]:
scale=0
def nor_diff( x, y):
    global scale
    result = np.sum(np.square(abs(x - y) / scale))
    return result

def additional_attri(a,b):
    global scale
    X=a.values
    y=b.values
    scale=4 * np.nanstd(X, axis=0)

    dis_metric = np.zeros((X.shape[0], X.shape[0]))
    for i in tqdm(range(X.shape[0])):
        inx = [i] * X.shape[0]
        for j, p in enumerate(inx):
            dis_metric[p, j] = nor_diff(X[p], X[j])

    for i, j in enumerate(range(X.shape[0])):
        dis_metric[i, j] = 1000000
    num_neighbor = 5

    top = np.argpartition(dis_metric, num_neighbor, axis=1)[:, :num_neighbor]
    #dis_neighbor = dis_metric[np.arange(dis_metric.shape[0])[:, None], top]  # compute neighbors for every sample


    count_neighbor = np.zeros((X.shape[0], num_neighbor+2))  # count the types of neighbors and decide type
    for i in tqdm(range(count_neighbor.shape[0])):
        count_neighbor[i, :num_neighbor] = y[top[i, :]]
        #count how many same labels with y[i]
        count=len(np.where(count_neighbor[i, :num_neighbor]==y[i])[0])
        count_neighbor[i, num_neighbor]=5-count
        #count_neighbor[i, num_neighbor] = abs(5 * y[i] -np.sum(count_neighbor[i, :num_neighbor]))
        if count_neighbor[i, num_neighbor] == 5:
            count_neighbor[i, num_neighbor+1] = 0.1
        elif count_neighbor[i, num_neighbor] == 4:
            count_neighbor[i, num_neighbor+1] = 0.25
        elif count_neighbor[i, num_neighbor] == 3:
            count_neighbor[i, num_neighbor+1] = 0.4
        elif count_neighbor[i, num_neighbor] == 2:
            count_neighbor[i, num_neighbor+1] = 0.6
        elif count_neighbor[i, num_neighbor] == 1:
            count_neighbor[i, num_neighbor+1] = 0.8
        else:
            count_neighbor[i, num_neighbor+1] = 1
    a['degree'] = count_neighbor[:, num_neighbor+1]
    return a,b

RACOG

In [6]:
from racog import RACOG
def rac(X_train,y_train):
    racog = RACOG(discretization='caim', categorical_features='auto',
                warmup_offset=100, lag0=20, n_iter='auto',
                continous_distribution='normal', random_state=None,
                alpha=0.6, L=0.5, threshold=5, eps=10E-5, verbose=2, n_jobs=-1)
    x_ra,y_ra=racog.fit_resample(X_train, y_train)
    x_ra=pd.DataFrame(x_ra,columns=X_train.columns)
    y_ra=pd.DataFrame(y_ra)
    return x_ra,y_ra

Boruta

In [7]:
from boruta import BorutaPy
def boruta_feature_selection(X, Y):  # X_test can be added here
    """
    Selecting promising features with boruta algorithm
    :param X: data features
    :param Y: data labels
    :return: feature vector, list of most selected features
    """
    smo = BorderlineSMOTE(random_state=42,sampling_strategy='auto')
    X, Y = smo.fit_resample(X, Y)
    rf = RandomForestClassifier(n_jobs=-1,
                                n_estimators=1000)

    feat_selector = BorutaPy(rf,
                             n_estimators=1000,
                             alpha=0.05,
                             max_iter=10, #here
                             verbose=2)

    #feat_selector.fit(X, Y)
    feat_selector.fit(X.values, Y)

    try:
        k = 0
        features = X.columns  # add here feature names
        features_list = []
        features_importance_list = []

        for i in feat_selector.support_:
            if i:
                features_list.append(features[k])
                features_importance_list.append(feat_selector.ranking_[k])
            k = k + 1

        features_list = [x for _, x in sorted(zip(features_importance_list, features_list))]
    except:
        features_list = ['no feature names given']

    X_boruta = feat_selector.transform(X.values)

    return features_list

Automated Tuned RF

In [2]:
def atrf(X,Y):

    # define level and initialize constructor
    subpath = "baseline"
    ct = Constants(subpath) #the pre-defination

    # initialise logger
    name_experiment = time.strftime("%Y%m%d_%H%M%S") + "_" + str(ct.DATASETS[0])
    logger = init_logging(ct.PATH, name_experiment)

    # Loading data
    logger.info("Loading data...")



    col=X.columns

    var = VarianceThreshold(threshold=1.0)
    X = var.fit_transform(X)
    select_name_index = var.get_support(indices=True)  # 留下特征的索引值，list格式
    select_name = []
    for i in select_name_index:
        select_name.append(col[i])
    X=pd.DataFrame(X,columns=select_name)
    
    col=X.columns
    X = preprocessing.scale(X)
    X=pd.DataFrame(X,columns=col)


    logger.info(f"Data --> X_shape: {X.shape}, Y_shape: {Y.shape}")
    
    dictionary=phcp(
        PATH=ct.PATH,
        name_experiment=name_experiment,
        logger=logger,
        subpath=subpath,
        X=X.values,
        Y=Y["label"].values,
    )
    return dictionary
    

Run pipeline

In [ ]:
if __name__ == '__main__':
    filename="./data_save/1_4_0.5"

    X=pd.read_csv(filename+"_X.csv",index_col=0)
    y=pd.read_csv(filename+"_y.csv",index_col=0)
    X=X.head(1000)
    y=y.head(1000)
    col=X.columns
    var = VarianceThreshold(threshold=1.0)
    X = var.fit_transform(X)
    select_name_index =var.get_support(indices=True)
    new_col=col[select_name_index]
    X = preprocessing.scale(X)
    X=pd.DataFrame(X,columns=new_col)
    X_feature=boruta_feature_selection(X,y["label"])
    X=X[X_feature]
    X,y["label"]=additional_attri(X,y["label"])
    rfdic=atrf(X,y)
    y=y["label"]

    kf = KFold(n_splits=10,shuffle=True)
    f1_score=[]
    acc=[]
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
            X_train = X.values[train_index]
            y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
            X_test = X.values[test_index]
            y_test = y[test_index]  # See comment on ravel and  y_train

            smo = BorderlineSMOTE()
            X_train_oversampled, y_train_oversampled = smo.fit_resample(X_train, y_train)
            model = RandomForestClassifier(n_estimators=int(rfdic["n_estimators"]),max_depth=int(rfdic["max_depth"]),max_features=(rfdic["max_depth"]),min_samples_leaf=int(rfdic["min_samples_leaf"]),min_samples_split=int(rfdic["min_samples_split"]))  # Choose a model here
            model.fit(X_train_oversampled, y_train_oversampled )  
            y_pred = model.predict(X_test)
            print(f'For fold {fold}:')
            
            print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
            print("F1 Score:",metrics.f1_score(y_test, y_pred, average='weighted'))
            acc.append(metrics.accuracy_score(y_test, y_pred))
            f1_score.append(metrics.f1_score(y_test, y_pred, average='weighted'))
    print("The average Accuracy is: ",np.mean(acc))
    print("The average F1 Score is: ",np.mean(f1_score))